# Hyperbolic trajectory
----

In [1]:
import numpy as np
from numpy import cos, sin, cosh, sinh, tanh, array,pi, exp
from numpy.linalg import norm,solve
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import scipy.linalg
from System import MakeSystem, FindHyperbolicTrajectory, MakeLinearisedSolution

In [2]:
k = 0.5
ForcingParameters = array([.1,1,.2,5])
f,lminus,lplus,G = MakeSystem(k,ForcingParameters)

In [3]:
XQ = MakeLinearisedSolution(ForcingParameters,lplus,lminus)
tmin,tmax = -3,3

N = 200
Xhyp,t,Xhyp0 = FindHyperbolicTrajectory(f,lminus,lplus,XQ,tmin,tmax,N)

 after 5 iterations an estimate with error of F =2.061567820263631e-12 and delta = 1.3794493178275686e-09was produced


In [ ]:
fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot(projection='3d')

ax.plot(X[:,0], X[:,1], t,linestyle='-',  linewidth=3, marker='o', label="Non-linear hyperbolic trajectory, quasi-periodic")
ax.plot( X0[:,0], X0[:,1],t,linestyle='--', linewidth=2, marker='x', label="Linear periodic trajectory, quasi-periodic")
plt.legend(fontsize=14)
ax.set_xlabel("$x$", fontsize=14)
plt.xlim(-1, 1)
plt.ylim(-1, 1)
ax.set_ylabel("$y$", fontsize=14)
ax.set_zlabel("$t$", fontsize=14)
plt.show()

plt.plot(t, X[:,0], color='blue', linestyle='-',  linewidth=3, marker='o', label="Non-linear hyperbolic trajectory")
plt.plot(t, X0[:,0], color='red',  linestyle='--', linewidth=2, marker='x', label="Linear periodic trajectory")
plt.title("Hyperbolic trajectories $x$ component, quasi-periodic", fontsize=20)
plt.xlabel("$t$", fontsize=18)
plt.ylabel("$x$", fontsize=18)
plt.legend(fontsize=6)
plt.show()
plt.plot(t, X[:,1], color='blue', linestyle='-',  linewidth=3, marker='o', label="Non-linear hyperbolic trajectory")
plt.plot(t, X0[:,1], color='red',  linestyle='--', linewidth=2, marker='x', label="Linear periodic trajectory")
plt.title("Hyperbolic trajectories $y$ component, quasi-periodic", fontsize=20)
plt.xlabel("$t$", fontsize=18)
plt.ylabel("$y$", fontsize=18)
plt.legend(fontsize=6)
plt.show()

In [ ]:
from StableManifoldTrajectories import FindStableTrajectory

In [ ]:
tmin,tmax = -3,3
t = np.linspace(tmin,tmax,N)

N = 200
J = int(N//2)
p=1
tmin = -3
X,tnew,X0 = FindStableTrajectory(f,lminus,lplus,tmin,tmax,N,p,J,Xhyp)

In [ ]:
X.shape

In [ ]:
fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot(projection='3d')
ax.plot(Xhyp[:,0], Xhyp[:,1], t,linestyle='--',  linewidth=3, marker='o', label="Non-linear hyperbolic trajectory")

ax.plot(X[:,0], X[:,1], tnew,linestyle='-',  linewidth=3, marker='o', label="Non-linear Stable manifold trajectory")
ax.plot( X0[:,0], X0[:,1],tnew,linestyle='--', linewidth=2, marker='x', label="Linear stable manifold trajectory")
plt.legend(fontsize=14)
ax.set_xlabel("$x$", fontsize=14)
plt.xlim(-1, 1)
plt.ylim(-1, 1)
ax.set_ylabel("$y$", fontsize=14)
ax.set_zlabel("$t$", fontsize=14)
plt.show()
plt.plot(t, Xhyp[:,0], color='green', linestyle='-',  linewidth=3, marker='o', label="Non-linear hyperbolic trajectory")

plt.plot(tnew, X[:,0], color='blue', linestyle='-',  linewidth=3, marker='o', label="Non-linear stable manifold trajectory")
plt.plot(tnew, X0[:,0], color='red',  linestyle='--', linewidth=2, marker='x', label="Linear stable manifold trajectory")
plt.title("Hyperbolic trajectories $x$ component, quasi-periodic", fontsize=20)
plt.xlabel("$t$", fontsize=18)
plt.ylabel("$x$", fontsize=18)
plt.legend(fontsize=6)
plt.show()
plt.plot(t, Xhyp[:,1], color='green', linestyle='-',  linewidth=3, marker='o', label="Non-linear hyperbolic trajectory")

plt.plot(tnew, X[:,1], color='blue', linestyle='-',  linewidth=3, marker='o', label="Non-linear stable manifold  trajectory")
plt.plot(tnew, X0[:,1], color='red',  linestyle='--', linewidth=2, marker='x', label="Linear stable manifold trajectory")
plt.title("stable manifold  trajectories $y$ component", fontsize=20)
plt.xlabel("$t$", fontsize=18)
plt.ylabel("$y$", fontsize=18)
plt.legend(fontsize=6)
plt.show()

# Stable manifold data collection
---

In [4]:
from StableManifoldTrajectories import FindStableTrajectory,FindUnstableTrajectory
import scipy.io
import scipy.interpolate
from scipy.interpolate import RegularGridInterpolator
from scipy.interpolate import Rbf
from scipy.interpolate import LinearNDInterpolator
from scipy.interpolate import interp1d


In [5]:
from numpy import nan
def ySM(p,s):
    for l in range(0,len(tindices)):
        if t[tindices[l]] == s:
            return  interpyS[t[tindices[l]]](array([p]))
        elif t[l] < s and t[l+1]> s:
            t0 = t[tindices[l]] 
            t1 = t[tindices[l+1]]
            break
    if l< len(tindices) - 2:
        SM1 = interpyS[t0](array([p]))
        SM2 = interpyS[t1](array([p]))
        return SM1 + ((s - t0)/(t1-t0))*(SM2-SM1)
    else: 
        return nan
def xUM(q,s):
    for l in range(0,len(tindices)):
        if t[tindices[l]] == s:
            return  interpxU[t[tindices[l]]](array([q]))
        elif t[l] < s and t[l+1]> s:
            t0 = t[tindices[l]] 
            t1 = t[tindices[l+1]]
            break
    if l< len(tindices) - 2:
        UM1 = interpxU[t0](array([q]))
        UM2 = interpxU[t1](array([q]))
        return UM1 + ((s - t0)/(t1-t0))*(UM2-UM1)
    else: 
        return nan
    

In [6]:
def makeinterpyS(P,yvals,tindices):
    interpyS = {  t[tindices[0]]: interp1d(P,yvals[:len(P)] )}
    for l in range(1,len(tindices)):
        interpyS[t[tindices[l]]] = interp1d(P,yvals[l*len(P):(l+1)*len(P)],kind = 'linear')
    return interpyS

def makeinterpxU(Q,xvals,tindices):
    interpxU = {  t[tindices[0]]: interp1d(Q,xvals[:len(Q)] )}
    for l in range(1,len(tindices)):
        interpxU[t[tindices[l]]] = interp1d(Q,xvals[l*len(Q):(l+1)*len(Q)],kind = 'linear')
    return interpxU

In [7]:
P = np.linspace(-2,2,10)
Q =np.linspace(-2,2,10)
tmin,tmax = -3,3
t = np.linspace(tmin,tmax,N)
N = 200
trangestart = N//5
trangeend = 4*trangestart
tindices = np.arange(int(N//5),int(4*N//5),10)
#tindices = np.array([3*N//4])
trange = t[tindices]
yvals = np.zeros(len(P)*len(tindices))
xvals = np.zeros(len(Q)*len(tindices))

l=0
%matplotlib qt
fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot(projection='3d')
ax.plot(Xhyp[:,0], Xhyp[:,1], t,linestyle='--', color = 'red', linewidth=3, marker='o', label="Non-linear hyperbolic trajectory")
for J in tindices:
    for i in range(0,len(P)):
        p = P[i]
        X,tnew,X0,Jnew = FindStableTrajectory(f,lminus,lplus,tmin,tmax,N,p,J,Xhyp)
       # if l==0:
        #    ax.plot(X[Jnew:,0], X[Jnew:,1], tnew[Jnew:],linestyle='-', color = 'blue', linewidth=1, marker='o', label="Non-linear Stable manifold trajectory")
        #else:
        #    ax.plot(X[Jnew:,0], X[Jnew:,1], tnew[Jnew:],linestyle='-', color = 'blue', linewidth=1, marker='o')
        yvals[l] = X[Jnew][1]   # output of function applied to (trange[J],P[i])
        q = Q[i]
        X,tnew,X0,Jnew = FindUnstableTrajectory(f,lminus,lplus,tmin,tmax,N,q,J,Xhyp)
     #   if l==0:
    #        ax.plot(X[Jnew:,0], X[Jnew:,1], tnew[Jnew:],linestyle='-', color = 'blue', linewidth=1, marker='o', label="Non-linear Stable manifold trajectory")
     #   else:
     #       ax.plot(X[Jnew:,0], X[Jnew:,1], tnew[Jnew:],linestyle='-', color = 'blue', linewidth=1, marker='o')
        xvals[l] = X[Jnew][0]   # output of function applied to (trange[J],P[i])
        l+=1
        if int(100*l/(len(tindices)*len(P)))%5 ==0:
            print(str(int(100*l/(len(tindices)*len(P)))) + "% complete")
        

interpyS = makeinterpyS(P,yvals,tindices)
interpxU = makeinterpxU(Q,xvals,tindices)

Pinterp = np.linspace(P[0]+0.1,P[-1]-0.1,15)
Qinterp = np.linspace(Q[0]+0.1,Q[-1]-0.1,15)
for J in tindices:
    Yvals =0*Pinterp
    Xvals = 0*Qinterp
    for i in range(0,len(Pinterp)):
        Yvals[i] = ySM(Pinterp[i],t[J])
    ax.plot(Pinterp[:]+Xhyp[J][0],Yvals[:] ,t[J], color = 'green', linewidth=3, marker='x')
    #plt.legend(fontsize=14)
    for i in range(0,len(Qinterp)):
        Xvals[i] = xUM(Qinterp[i],t[J])
    ax.plot(Xvals[:],Qinterp[:] + Xhyp[J][1],t[J], color = 'blue', linewidth=3, marker='x')
    
    ax.set_xlabel("$x$", fontsize=14)
    #plt.xlim(-1, 1)
    #plt.ylim(-1, 1)
    ax.set_ylabel("$y$", fontsize=14)
    ax.set_zlabel("$t$", fontsize=14)
    plt.show()
    
    
    

 after 40 iterations an estimate with error of F =3.3951744666862415e-09 and delta = 7.65017843831957e-08was produced
 after 46 iterations an estimate with error of F =4.185746225223417e-09 and delta = 7.15245815253589e-08was produced
 after 18 iterations an estimate with error of F =1.6598116654863834e-09 and delta = 4.318153833506062e-08was produced
 after 28 iterations an estimate with error of F =4.088400283038432e-09 and delta = 8.560409722844022e-08was produced
 after 6 iterations an estimate with error of F =1.7233684928754432e-12 and delta = 1.0012803294529787e-09was produced
 after 6 iterations an estimate with error of F =8.680562266866616e-13 and delta = 5.707235182400308e-10was produced
5% complete
 after 17 iterations an estimate with error of F =1.6641284233701126e-09 and delta = 4.3318608649322904e-08was produced
 after 32 iterations an estimate with error of F =3.0262601097679966e-09 and delta = 6.045936501114742e-08was produced
 after 32 iterations an estimate with err

In [13]:
np.save("P.npy",P)
np.save("yvals.npy",yvals)
np.save("tindices.npy",tindices)
np.save("t.npy",t)
np.save("Xhyp.npy",Xhyp)
np.save("Q.npy",Q)
np.save("xvals.npy",xvals)
np.save("ForcingParameters.npy",ForcingParameters)

In [ ]:
P = np.load("P.npy")
yvals = np.load("yvals.npy")
tindices = np.load("tindices.npy")
t = np.load("t.npy")
Xhyp = np.load("Xhyp.npy")

## Plots
----

Plot of  manifolds in stable/unstable coordinates.

In [8]:
%matplotlib qt
fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot(projection='3d')
ax.plot(Xhyp[:,0], Xhyp[:,1], t,linestyle='--', color = 'green', linewidth=3, marker='o', label="Hyperbolic trajectory")
Pinterp = np.linspace(P[0]+0.1,P[-1]-0.1,15)
Qinterp = np.linspace(Q[0]+0.1,Q[-1]-0.1,15)
for J in tindices:
    Yvals =0*Pinterp
    Xvals = 0*Qinterp
    for i in range(0,len(Pinterp)):
        Yvals[i] = ySM(Pinterp[i],t[J])
    if J == tindices[0]:
        ax.plot(Pinterp[:]+Xhyp[J][0],Yvals[:] ,t[J], color = 'blue', linewidth=3, marker='x',label = "Stable manifold")
    else:
        ax.plot(Pinterp[:]+Xhyp[J][0],Yvals[:] ,t[J], color = 'blue', linewidth=3, marker='x')
    
    for i in range(0,len(Qinterp)):
        Xvals[i] = xUM(Qinterp[i],t[J])
        
    if J == tindices[0]:
        ax.plot(Xvals[:],Qinterp[:] + Xhyp[J][1],t[J], color = 'red', linewidth=3, marker='x', label = "Unstable manifold")
    else:
        ax.plot(Xvals[:],Qinterp[:] + Xhyp[J][1],t[J], color = 'red', linewidth=3, marker='x')
    
    plt.legend(fontsize = 14)
    ax.set_xlabel("$x$", fontsize=14)
    #plt.xlim(-1, 1)
    #plt.ylim(-1, 1)
    ax.set_ylabel("$y$", fontsize=14)
    ax.set_zlabel("$t$", fontsize=14)
    plt.show()

Time slice in stable/unstable coordinates

In [9]:
fig = plt.figure(figsize=(15,15))
Pinterp = np.linspace(P[0]+0.1,P[-1]-0.1,15)
Qinterp = np.linspace(Q[0]+0.1,Q[-1]-0.1,15)
J= tindices[0]
Yvals =0*Pinterp
for i in range(0,len(Pinterp)):
    Yvals[i] = ySM(Pinterp[i],t[J])
plt.plot(Pinterp[:]+Xhyp[J][0],Yvals[:], color = 'blue', linewidth=3, marker='x', label = "stable manifold")
plt.plot(Xhyp[J][0],Xhyp[J][1],marker = 'o',label ="hyp traj",color = 'green')
for i in range(0,len(Qinterp)):
        Xvals[i] = xUM(Qinterp[i],t[J])
plt.plot(Xvals[:],Qinterp[:] + Xhyp[J][1],t[J], color = 'red', linewidth=3, marker='x',label = 'unstable manifold')
    
plt.legend(fontsize=14)
plt.xlabel("$x$", fontsize=14)
    #plt.xlim(-1, 1)
    #plt.ylim(-1, 1)
plt.ylabel("$y$", fontsize=14)
plt.show()

Time slice in original coordinates.

In [14]:
#Changing coordinates back to original
Minv = (lplus - lminus)**(-1)*np.array([[lplus,-1],[-lminus,1]])
M = np.linalg.inv(Minv)
fig = plt.figure(figsize=(15,15))
Pinterp = np.linspace(P[0]+0.1,P[-1]-0.1,15)
Qinterp = np.linspace(Q[0]+0.1,Q[-1]-0.1,15)
J= tindices[0]
Yvals =0*Pinterp
for i in range(0,len(Pinterp)):
    Yvals[i] = ySM(Pinterp[i],t[J])
    
# changing coords    
    
plt.plot(M.dot(np.array([Pinterp[:]+Xhyp[J][0],Yvals[:]]))[0], M.dot(np.array([Pinterp[:]+Xhyp[J][0],Yvals[:]]))[1]   ,         color = 'blue', linewidth=3, marker='x', label = "Stable manifold")
plt.plot(M.dot(Xhyp[J])[0],M.dot(Xhyp[J])[1],marker = 'o',label ="hyp traj",color = 'green')
for i in range(0,len(Qinterp)):
        Xvals[i] = xUM(Qinterp[i],t[J])
plt.plot(M.dot(np.array([Xvals[:],Qinterp[:] + Xhyp[J][1]]))[0],M.dot(np.array([Xvals[:],Qinterp[:] + Xhyp[J][1]]))[1], color = 'red', linewidth=3, marker='x',label = 'Unstable manifold')
    
plt.legend(fontsize=14)
plt.xlabel("$x$", fontsize=14)
    #plt.xlim(-1, 1)
    #plt.ylim(-1, 1)
plt.ylabel("$y$", fontsize=14)
plt.show()

Manifolds in original coordinates.

In [13]:
%matplotlib qt
fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot(projection='3d')
ax.plot(Xhyp[:,0], Xhyp[:,1], t,linestyle='--', color = 'green', linewidth=3, marker='o', label="Non-linear hyperbolic trajectory")
Pinterp = np.linspace(P[0]+0.1,P[-1]-0.1,15)
Qinterp = np.linspace(Q[0]+0.1,Q[-1]-0.1,15)
for J in tindices:
    Yvals =0*Pinterp
    Xvals = 0*Qinterp
    for i in range(0,len(Pinterp)):
        Yvals[i] = ySM(Pinterp[i],t[J])
    if J == tindices[0]:
        ax.plot(M.dot(np.array([Pinterp[:]+Xhyp[J][0],Yvals[:]]))[0],M.dot(np.array([Pinterp[:]+Xhyp[J][0],Yvals[:]]))[1] ,t[J], color = 'blue', linewidth=3, marker='x',label = 'Stable manifold')
    else:
        ax.plot(M.dot(np.array([Pinterp[:]+Xhyp[J][0],Yvals[:]]))[0],M.dot(np.array([Pinterp[:]+Xhyp[J][0],Yvals[:]]))[1] ,t[J], color = 'blue', linewidth=3, marker='x')
    
    for i in range(0,len(Qinterp)):
        Xvals[i] = xUM(Qinterp[i],t[J])
    if J ==tindices[0]:
        ax.plot(M.dot(np.array([Xvals[:],Qinterp[:] + Xhyp[J][1]]))[0],M.dot(np.array([Xvals[:],Qinterp[:] + Xhyp[J][1]]))[1],t[J], color = 'red', linewidth=3, marker='x',label = "Unstable manifold")
    else:
        ax.plot(M.dot(np.array([Xvals[:],Qinterp[:] + Xhyp[J][1]]))[0],M.dot(np.array([Xvals[:],Qinterp[:] + Xhyp[J][1]]))[1],t[J], color = 'red', linewidth=3, marker='x')
    ax.set_xlabel("$x$", fontsize=14)
    #plt.xlim(-1, 1)
    #plt.ylim(-1, 1)
    plt.legend()
    ax.set_ylabel("$y$", fontsize=14)
    ax.set_zlabel("$t$", fontsize=14)
    plt.show()

# Heat map
----


In [15]:
# functions to determine if it passes over
f = lambda t,y : np.array([y[1], np.tanh(y[0])*np.cosh(y[0])**(-2) - k*y[1]]) # vector field in original coordinates

def event(t,y):
    return y[0]**2 - 3.5**2
event.terminal = True
# functions to determine if it passes over
def passOver(y0):
    res = solve_ivp (f, [0,500], y0 , method = "RK45" , events=[event],max_step =0.05 , rtol = 1e-8 )
    y = res .y. transpose()
    t = res .t
    if y[-1,0] > 3:
        return 1
#    if y[-1,0] <-3:
#        return -1
    else:
        return 0


#creating and storing the data.

x = np.meshgrid(np.linspace(-3, 3, 40),   np.linspace(-3,3, 40),indexing= 'ij')
z = np.zeros((np.shape(x)[1],np.shape(x)[2]))
for i in range(0,np.shape(x)[1]):
    for j in range(0,np.shape(x)[2]):
        y0 = np.array([x[0][i,j],x[1][i,j]])
        z[i,j] = passOver(y0)
        
np.save("initial conditions.npy" , x)
np.save("Points moving over.npy", z)

In [16]:
plt.contourf(x[0][:],x[1][:],z)
plt.colorbar()
plt.xlabel("x", fontsize=14)
plt.ylabel("y", fontsize=14)
plt.show()